# Upload 
> Notes on Writing and Flashing firmware using an IPYNB

- prettify: true

<img src='/images/covers/upload1.webp'>

<details>
<summary>

## Optional - Setup Localhost on Colabs

</summary>

Run this in the local terminal:

In [ ]:
jupyter notebook \ \
  --NotebookApp.allow_origin='https://colab.research.google.com' \ \
  --port=8888 \ \
  --NotebookApp.port_retries=0 \ \

'Then hit the CONNECT dropdown on the top right of this screen to Connect to the Localhost 

</details>
<details open>
<summary>

## Upload Function

</summary>

This is the upload script:

In [1]:
#collapse_input #hide_output
import subprocess
from IPython.display import display, clear_output
import os

def upload_code(code): 
    # Save the code to a file 
    with open('src/main.cpp', 'w') as f:
        f.write(code) 

    # Compile the code
    cmd = 'C:\\Users\\carlos\\.platformio\\penv\\Scripts\\platformio.exe run --target upload -v'
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    show = False

    # Wait for the words "Serial port" to appear in the output
    while True:
        output = process.stdout.readline().decode()
        if output == '' and process.poll() is not None:
            break
        if 'Serial port' in output:
            show = True 
        if show:
            display(output.strip())

    # Get the output and errors after the process has completed
    output, error = process.communicate()
    # print(output.decode())
    # print(error.decode())

    # Clear the output once the process has completed
    clear_output()

def read_and_upload_code(file_path): 
    with open(file_path, 'r') as f:
        code = f.read() 
        upload_code(code)

directory = os.path.join(os.getcwd(), 'samples')
files = os.listdir(directory)
for file in files:
    ''
    # print(file)

</details>
<details open>
<summary>

## Uploading

</summary>

This is the uplaod fn in use:

In [4]:
# file = 'a_FastLedFTT.cpp'
file = 'analogSoundSampleSketch.cpp'
# file = 'gpt4_v4.cpp'

In [5]:
read_and_upload_code(directory+'/'+file)

</details>
<details open>
<summary>

## Serial Logging

</summary>

This is a simple example to read the serial output

In [6]:
#collapse_input #hide_output
import serial
from IPython.display import clear_output

ser = serial.Serial()
ser.baudrate = 9600
ser.port = 'COM8'
ser.open()
ser.flushInput()

try:
    while True:
        inpt = ser.readline()
        print(inpt.decode('utf-8').strip())
except KeyboardInterrupt:
    print("Keyboard Interrupt. Exiting...")
finally: 
    clear_output()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3: invalid start byte

</details>
<details open>
<summary>

## Serial Plotting

</summary>

this is a more complex example

In [ ]:
#collapse_input #hide_output
import serial
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import csv
import time

ser = serial.Serial()
ser.baudrate = 9600
ser.port = 'COM8'
ser.open()
ser.flushInput()

# Create an empty list to store the data
data = [0] * 100

# Create the plot
plt.ion() # Turn on interactive mode
fig, ax = plt.subplots()
line, = ax.plot(data)
ax.set_title("Random Data")
ax.set_xlabel("Data Point")
ax.set_ylabel("Value")

try:
    while True:
        # Read a line of data from the serial port
        inpt = ser.readline().decode('utf-8').strip()

        # Parse the data and append it to the list
        try:
            value = int(inpt)
            data.pop(0)  # Remove the first value from the list
            data.append(value)  # Append the new value to the end of the list
        except IndexError:
            pass

        # Update the plot
        line.set_ydata(data)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw()
        fig.canvas.flush_events()

        # Display the plot
        clear_output(wait=True)
        display(fig)

        # save the output 
        with open("test_data.csv","a", newline="") as f:
            writer = csv.writer(f,delimiter=",")
            writer.writerow([time.time(),value])

except KeyboardInterrupt:
    print("Keyboard Interrupt. Exiting...")
finally:
    ser.close()
    clear_output()


</details>